<a href="https://colab.research.google.com/github/Schwaldlander/DomainNameSuggest/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a project on domain name suggestion.

Proposing a suitable Domain Name is a tricky assignment for entrepreneurs. Clarity, Pronunciation, Popular Reception, Cultural Implications, trademark laws and regulations shall be taken into account.  

Targets include:
1. Reproducible Performance with Model Version Tracking
2. Runnable evaluation framework that works across all model iterations
3. **Optional**: Deploy selected model as API endpoint


General Setting: GPU required

# First Step: Create a small, diverse synthetic dataset for domain-name suggestion tasks

It includes:
1) Briefs (JSONL)

2) Candidates with labels (JSONL)

3) Pairwise preference judgments (JSONL)

4) A README-style methodology (Markdown)


# Synthetic Dataset for Domain Name Suggestions

**Generated:** {datetime.utcnow().isoformat()}Z

## Files
- `domain_briefs.jsonl` — Diverse briefs (industry, tone, keywords, constraints, complexity)
- `domain_candidates.jsonl` — Candidate suggestions with scores, pass/fail, safety flags
- `domain_pairwise.jsonl` — Synthetic pairwise preferences for DPO/IPO

## Diversity Coverage
- **Business types**: fintech, eco cosmetics, B2B AI, coffee roaster, tutoring (FR), dev tools (DE), travel (ES), wellness, home IoT, climate nonprofit, JP stationery (translit), AR food delivery (translit), pet supplements, outdoor rentals, kids coding.
- **Languages/scripts**: EN, FR, DE, ES (Latin). JP/AR represented via **Latin transliteration** to avoid IDN in this first version.
- **Complexity levels**: basic, moderate, advanced (randomly assigned) indicating constraints richness and prompt realism.

## Methodology
1. **Brief Construction**: For each business type, we define language, tone, keywords, and constraints:
   - `max_len` (10/12/14), `allowed_tlds` (domain-appropriate),
   - forbid digits/hyphens, ASCII-only for v1 (IDN can be added later).
2. **Candidate Generation**: Nonce-word generator from a curated syllable bank creates pronounceable, brandable strings. We avoid real brands or adult/illegal terms.
3. **Safety & Constraints**: We inject a small fraction of *intentionally flawed* candidates (digits, hyphens, or trademark-like typosquats such as `go0gle-...`) to train and evaluate filters. No explicit harmful content is included.
4. **Weak Labels**: Each candidate has pseudo-scores (`brandability`, `brevity`, `keyword_fit`) for reranking studies. Replace with human ratings over time.
5. **Pairwise Preferences**: Winners are sampled from higher-scored, constraint-passing candidates; losers from lower-scored/flagged ones. This supports preference optimization (DPO/IPO/KTO).
6. **Intended Use**: Bootstrapping a generation→filter→rerank pipeline and automated tests. For production, add multilingual scripts, IDN/homograph checks, and human review.
7. **Ethics & Safety**: The dataset purposefully avoids generating or normalizing harmful categories (hate, sexual content, illegal goods/services, self-harm, extremist content). It includes negative examples only in the form of benign constraint violations and obvious trademark-like typosquats to test refusal/filters.

## Schemas
### Brief
```json
{{"brief_id":"uuid","title":"string","language":"en","script":"Latin","tone":"string","keywords":["k1","k2"],"constraints":{{"max_len":12,"allowed_tlds":[".com",".io"],"forbid_digits":true,"forbid_hyphens":true,"ascii_only":true}},"complexity":"basic|moderate|advanced","notes":"string"}}
```


### Candidate
```json
{{"candidate_id":"uuid","brief_id":"uuid","domain":"brevexa.com","rationale":"string","scores":{{"brandability":0.85,"brevity":0.9,"keyword_fit":0.7}},"passes_constraints":true,"safety":{{"flagged":false,"reasons":[]}}}}
```



In [1]:
import json, random, uuid, textwrap, os
from datetime import datetime
import pandas as pd


In [2]:
# google mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

import json, random, uuid, os, re
from datetime import datetime
import pandas as pd


random.seed(42)

# -------------------------
# Utilities
# -------------------------
def uid():
    return str(uuid.uuid4())

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

OUT_DIR = "/content/drive/MyDrive/domain_suggest/data"
ensure_dir(OUT_DIR)

# -------------------------
# 1) Briefs
# -------------------------
business_catalog = [
    # (title, keywords, tone, tlds, language, script, notes)
    ("Fintech payments wallet", ["pay", "wallet", "secure"], "premium, trustworthy", [".com",".io",".pay"], "en", "Latin", ""),
    ("Eco-friendly cosmetics", ["vegan", "plant", "glow"], "gentle, natural", [".com",".beauty",".shop"], "en", "Latin", ""),
    ("B2B AI analytics", ["insight", "metrics", "predict"], "modern, technical", [".ai",".io",".com"], "en", "Latin", ""),
    ("Artisanal coffee roaster", ["beans","roast","origin"], "craft, warm", [".com",".coffee",".shop"], "en", "Latin", ""),
    ("Online French tutoring", ["cours","langue","coach"], "convivial, sérieux", [".fr",".com"], "fr", "Latin", ""),
    ("SaaS developer tools (DE)", ["code","build","deploy"], "prägnant, professionell", [".de",".dev",".io"], "de", "Latin", ""),
    ("Travel planning app (ES)", ["viaje","ruta","plan"], "amable, inspirador", [".es",".app",".com"], "es", "Latin", ""),
    ("Wellness & yoga studio", ["flow","breathe","calm"], "soothing, minimalist", [".com",".studio",".fit"], "en", "Latin", ""),
    ("Home automation IoT", ["smart","home","mesh"], "sleek, futuristic", [".com",".tech",".io"], "en", "Latin", ""),
    ("Nonprofit climate org", ["climate","earth","action"], "serious, hopeful", [".org",".earth",".com"], "en", "Latin", ""),
    ("Japanese stationery (JP translit)", ["pen","paper","kawaii"], "cute, refined", [".jp",".shop",".com"], "ja", "Latin", "Transliterated keywords only"),
    ("Arabic food delivery (translit)", ["souk","fresh","sah"], "friendly, reliable", [".com",".me",".app"], "ar", "Latin", "Transliterated keywords only"),
    ("Pet supplements DTC", ["pet","chew","boost"], "friendly, credible", [".com",".pet",".shop"], "en", "Latin", ""),
    ("Outdoor gear rental", ["camp","hike","rent"], "adventurous, practical", [".com",".outdoors",".rentals"], "en", "Latin", ""),
    ("Kids coding classes", ["code","kids","learn"], "playful, educational", [".com",".school",".academy"], "en", "Latin", ""),
]

complexity_levels = ["basic","moderate","advanced"]

def make_briefs(catalog):
    briefs = []
    for (title, keywords, tone, tlds, lang, script, notes) in catalog:
        briefs.append({
            "brief_id": uid(),
            "title": title,
            "language": lang,
            "script": script,
            "tone": tone,
            "keywords": keywords,
            "constraints": {
                "max_len": random.choice([10,12,14]),
                "allowed_tlds": tlds,
                "forbid_digits": True,
                "forbid_hyphens": True,
                "ascii_only": True
            },
            "complexity": random.choice(complexity_levels),
            "notes": f"Synthetic brief; availability not verified. {notes}".strip()
        })
    return briefs

briefs = make_briefs(business_catalog)

# -------------------------
# 2) Candidate generation
# -------------------------

# A small syllable bank to form pronounceable nonce words (harmless content only)
syllables = [
    "bre","ve","xa","no","va","ly","zo","ri","ta","lo","fi","ki","ra","ne","mi","do","tu","su","pla","tri","quo","zen","lum","sio",
    "meta","nex","ora","kiri","terra","flux","vanta","pleni","astra","omni","veri","cora","mira","luma","axi","primo","alto","vivo",
    "nori","lumi","kora","vexa","tava","moro","lino","nexa","pivo","dela","soma","trio"
]

def gen_nonce(max_len):
    # build pronounceable-ish string; reserve 3 chars for ".tld"
    for _ in range(80):
        parts = random.sample(syllables, k=random.choice([2,3]))
        name = "".join(parts).lower()
        name = re.sub(r'(.)\1{2,}', r'\1\1', name)  # compress 3+ repeats
        if len(name) <= max_len and name.isascii() and name.isalpha():
            return name
        # fallback: trim
        if len(name) > max_len:
            name = name[:max_len]
            if name.isalpha():
                return name
    return "novexa"

def make_candidates_for_brief(b, k=12):
    cands = []
    max_len = b["constraints"]["max_len"]
    allowed_tlds = b["constraints"]["allowed_tlds"]

    # functions that create intentionally *bad* examples (for training filters)
    def inj_digit(base): return base.replace("o","0") + random.choice(allowed_tlds)
    def inj_hyphen(base): return base + "-pro" + random.choice(allowed_tlds)
    def inj_trademark_like(base): return "go0gle-" + base + random.choice(allowed_tlds)

    bad_funcs = [inj_digit, inj_hyphen, inj_trademark_like]

    for i in range(k):
        base = gen_nonce(max_len)
        tld = random.choice(allowed_tlds)
        domain = f"{base}{tld}"
        rationale = f"Short coined word aligned to keywords ({', '.join(b['keywords'])}) and tone '{b['tone']}'."
        safety = {"flagged": False, "reasons": []}
        passes = True

        # inject one flawed sample per 6
        if (i+1) % 6 == 0:
            dom = random.choice(bad_funcs)(base)
            domain = dom

        # checks
        name_part = domain.split(".")[0]
        if len(name_part) > max_len:
            passes = False
        if any(ch.isdigit() for ch in domain):
            passes = False; safety["flagged"]=True; safety["reasons"].append("contains_digit")
        if "-" in domain:
            passes = False; safety["flagged"]=True; safety["reasons"].append("contains_hyphen")
        if domain in ["go0gle" ,"yah0o", "nazi"]:
            passes = False; safety["flagged"]=True; safety["reasons"].append("trademark_like")

        scores = {
            "brandability": round(random.uniform(0.6, 0.95),2),
            "brevity": round(max(0.3, 1 - len(name_part)/max(6, max_len)),2),
            "keyword_fit": round(random.uniform(0.55, 0.9),2)
        }
        cands.append({
            "candidate_id": uid(),
            "brief_id": b["brief_id"],
            "domain": domain,
            "rationale": rationale,
            "scores": scores,
            "passes_constraints": passes,
            "safety": safety
        })
    return cands

candidates = []
for b in briefs:
    candidates.extend(make_candidates_for_brief(b, k=12))

# -------------------------
# 3) Pairwise preferences (synthetic DPO/IPO data)
# -------------------------
pairwise = []
for b in briefs:
    bcands = [c for c in candidates if c["brief_id"] == b["brief_id"]]
    ranked = sorted(bcands, key=lambda x: (x["passes_constraints"], x["scores"]["brandability"]), reverse=True)
    tops = ranked[:4]
    bots = ranked[-4:]
    for a in tops:
        for d in bots:
            pairwise.append({
                "pair_id": uid(),
                "brief_id": b["brief_id"],
                "winner_candidate_id": a["candidate_id"],
                "loser_candidate_id": d["candidate_id"],
                "reason_codes": ["brandability","constraint_pass","safety_margin"]
            })

# -------------------------
# 4) Write files
# -------------------------
paths = {
    "briefs": os.path.join(OUT_DIR, "domain_briefs.jsonl"),
    "candidates": os.path.join(OUT_DIR, "domain_candidates.jsonl"),
    "pairwise": os.path.join(OUT_DIR, "domain_pairwise.jsonl"),
    "readme": os.path.join(OUT_DIR, "README_methodology.md"),
    "script": os.path.join(OUT_DIR, "generate_synthetic_dataset.py"),
}

# if path not exist
if not os.path.exists(paths["briefs"]):
  with open(paths["briefs"], "w", encoding="utf-8") as f:
    for b in briefs:
        f.write(json.dumps(b, ensure_ascii=False)+"\n")

if not os.path.exists(paths["candidates"]):
  with open(paths["candidates"], "w", encoding="utf-8") as f:
    for c in candidates:
        f.write(json.dumps(c, ensure_ascii=False)+"\n")

if not os.path.exists(path[])
with open(paths["pairwise"], "w", encoding="utf-8") as f:
    for p in pairwise:
        f.write(json.dumps(p, ensure_ascii=False)+"\n")



## Model Development & Iteration
• Baseline Model: Fine-tune initial open-source LLM. You can use common recipes for that.

• Improved Model(s): Address discovered issues through, i.e.:

o Dataset augmentation

o Different fine-tuning approaches (LoRA, full fine-tuning, etc.)

o Hyperparameter optimization

• Save and version all model checkpoints

In [5]:
!pip -q install "transformers>=4.43" "datasets>=2.19" "accelerate>=0.33" "peft>=0.12" "bitsandbytes>=0.43" "trl>=0.9" sentencepiece evaluate huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [6]:
from huggingface_hub import login
login()  # paste my HF token


In [7]:
import json, random, os, pandas as pd, datasets as ds
from sklearn.model_selection import train_test_split

DATA_DIR = "/content/drive/MyDrive/domain_suggest/data"  # put your files here
briefs_path = f"{DATA_DIR}/domain_briefs.jsonl"
cands_path  = f"{DATA_DIR}/domain_candidates.jsonl"
pairs_path  = f"{DATA_DIR}/domain_pairwise.jsonl"

# Build SFT samples: (brief) -> (JSON suggestions)
# We'll group candidates by brief and produce small top-k sets as targets.
def load_briefs(path):
    return [json.loads(x) for x in open(path, "r", encoding="utf-8").read().splitlines()]

def load_cands(path):
    return [json.loads(x) for x in open(path, "r", encoding="utf-8").read().splitlines()]

briefs = load_briefs(briefs_path)
cands  = load_cands(cands_path)

# Build one training example per brief: prompt = brief (YAML-ish), target = JSON with k suggestions
by_brief = {}
for c in cands:
    by_brief.setdefault(c["brief_id"], []).append(c)

samples = []
K = 6  # number of suggestions to train on
for b in briefs:
    group = by_brief.get(b["brief_id"], [])
    # choose valid, constraint-passing first; backfill with others if needed
    good = [x for x in group if x.get("passes_constraints", False) and not x["safety"]["flagged"]]
    pool = good if len(good) >= K else (good + [x for x in group if x not in good])
    sel = sorted(pool, key=lambda x: x["scores"]["brandability"], reverse=True)[:K]

    prompt = f"""You are a brand-safe domain name generator.
Follow the policy: refuse unsafe requests; output valid JSON schema only.

[BRIEF]
title: {b['title']}
language: {b['language']}
tone: {b['tone']}
keywords: {', '.join(b['keywords'])}
constraints:
  max_len: {b['constraints']['max_len']}
  allowed_tlds: {', '.join(b['constraints']['allowed_tlds'])}
  forbid_digits: {b['constraints']['forbid_digits']}
  forbid_hyphens: {b['constraints']['forbid_hyphens']}
  ascii_only: {b['constraints']['ascii_only']}
"""

    target = {
        "query_id": b["brief_id"],
        "suggestions": [
            {
                "domain": s["domain"].split(".")[0] + s["domain"][len(s["domain"].split(".")[0]):],
                "rationale": s["rationale"],
                "scores": s["scores"],
                "safety": s["safety"],
            } for s in sel
        ],
        "notes": ["Availability not verified"]
    }
    samples.append({"prompt": prompt.strip(), "response": json.dumps(target, ensure_ascii=False)})

train, test = train_test_split(samples, test_size=0.15, random_state=7)
val, test  = train_test_split(test, test_size=0.5, random_state=7)

dataset = ds.DatasetDict({
    "train": ds.Dataset.from_list(train),
    "validation": ds.Dataset.from_list(val),
    "test": ds.Dataset.from_list(test)
})
dataset


DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 12
    })
    validation: Dataset({
        features: ['prompt', 'response'],
        num_rows: 1
    })
    test: Dataset({
        features: ['prompt', 'response'],
        num_rows: 2
    })
})

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model
from transformers import DataCollatorForLanguageModeling, Trainer
import torch, os, json

BASE_MODEL = "Qwen/Qwen2.5-3B-Instruct"
OUTPUT_DIR = "/content/drive/MyDrive/domain_suggest/checkpoints/baseline_qlora"
os.makedirs(OUTPUT_DIR, exist_ok=True)

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True) #padding_side="left"
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_cfg,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
# IMPORTANT: prepare for k-bit training + gradient checkpointing




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

NameError: name 'prepare_model_for_kbit_training' is not defined

In [9]:
from peft import prepare_model_for_kbit_training
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False
model.gradient_checkpointing_enable()

peft_cfg = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],  # common names; adjusts per model
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_cfg)

# Format to dialogue-style prompt → response!
def format_example(ex):
    sys = "You generate brand-safe domain suggestions and strictly refuse unsafe requests. Output strict JSON."
    user = ex["prompt"]
    assistant = ex["response"]
    # simple chat template
    text = f"<|im_start|>system\n{sys}\n<|im_end|>\n<|im_start|>user\n{user}\n<|im_end|>\n<|im_start|>assistant\n{assistant}\n<|im_end|>"
    return {"input_ids": tok(text, truncation=True, max_length=2048, padding="max_length")["input_ids"]}

tokenized = dataset.map(format_example, remove_columns=dataset["train"].column_names, num_proc=1)
collator = DataCollatorForLanguageModeling(tok, mlm=False)


Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [10]:
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=2,
    logging_steps=20,
    eval_strategy="epoch",
    save_strategy="epoch",
    bf16=True,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    gradient_checkpointing=True,
    ddp_find_unused_parameters=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    data_collator=collator
)
trainer.train()

# Save PEFT checkpoint (LoRA weights) + tokenizer
trainer.save_model(OUTPUT_DIR)
tok.save_pretrained(OUTPUT_DIR)

Epoch,Training Loss,Validation Loss
1,No log,1.890589
2,No log,1.413511


('/content/drive/MyDrive/domain_suggest/checkpoints/baseline_qlora/tokenizer_config.json',
 '/content/drive/MyDrive/domain_suggest/checkpoints/baseline_qlora/special_tokens_map.json',
 '/content/drive/MyDrive/domain_suggest/checkpoints/baseline_qlora/chat_template.jinja',
 '/content/drive/MyDrive/domain_suggest/checkpoints/baseline_qlora/vocab.json',
 '/content/drive/MyDrive/domain_suggest/checkpoints/baseline_qlora/merges.txt',
 '/content/drive/MyDrive/domain_suggest/checkpoints/baseline_qlora/added_tokens.json',
 '/content/drive/MyDrive/domain_suggest/checkpoints/baseline_qlora/tokenizer.json')

In [11]:
from transformers import TextStreamer
streamer = TextStreamer(tok, skip_prompt=True, skip_special_tokens=True)

def gen(brief):
    prompt = f"""title: {brief['title']}
language: {brief['language']}
tone: {brief['tone']}
keywords: {', '.join(brief['keywords'])}
constraints:
  max_len: {brief['constraints']['max_len']}
  allowed_tlds: {', '.join(brief['constraints']['allowed_tlds'])}
  forbid_digits: {brief['constraints']['forbid_digits']}
  forbid_hyphens: {brief['constraints']['forbid_hyphens']}
  ascii_only: {brief['constraints']['ascii_only']}"""

    sys = "You generate brand-safe domain suggestions and strictly refuse unsafe requests. Output strict JSON only."
    text = f"<|im_start|>system\n{sys}\n<|im_end|>\n<|im_start|>user\n{prompt}\n<|im_end|>\n<|im_start|>assistant\n"
    inputs = tok(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=400, do_sample=True, temperature=0.7, top_p=0.9, streamer=streamer)
    print()  # newline

gen(random.choice(briefs))


{
  "suggestions": [
    "cute-paper-shop.jp",
    "paper-kawaii.com",
    "kawaii-pen.shop"
  ]
}



In [12]:
from copy import deepcopy
import math, numpy as np

sweep = [
  {"r": 8,  "lr": 2e-4, "epochs": 2},
  {"r": 16, "lr": 1e-4, "epochs": 2},
  {"r": 32, "lr": 8e-5, "epochs": 3},
]
results = []

for i, hp in enumerate(sweep, 1):
    out = f"/content/drive/MyDrive/domain_suggest/checkpoints/qlora_sweep_run{i}"
    cfg = deepcopy(peft_cfg)
    cfg.r = hp["r"]
    model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, quantization_config=bnb_cfg, device_map="auto", torch_dtype=torch.bfloat16)
    model = get_peft_model(model, cfg)

    args = TrainingArguments(
        output_dir=out,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=8,
        learning_rate=hp["lr"],
        num_train_epochs=hp["epochs"],
        logging_steps=25,
        eval_strategy="epoch",
        save_strategy="epoch",
        bf16=True,
        gradient_checkpointing=True,
        report_to="none"
    )
    trainer = Trainer(model=model, args=args, train_dataset=tokenized["train"], eval_dataset=tokenized["validation"], data_collator=collator)
    trainer.train()
    metrics = trainer.evaluate()
    results.append({"run": i, **hp, **metrics})

pd.DataFrame(results)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
pairs = [json.loads(x) for x in open(pairs_path, "r", encoding="utf-8").read().splitlines()]
cand_map = {c["candidate_id"]: c for c in cands}
brief_map = {b["brief_id"]: b for b in briefs}

def dpo_rows(pairs):
    rows = []
    for p in pairs:
        b = brief_map[p["brief_id"]]
        w = cand_map[p["winner_candidate_id"]]
        l = cand_map[p["loser_candidate_id"]]
        prompt = f"""Provide JSON suggestions for this brief.

title: {b['title']}
language: {b['language']}
tone: {b['tone']}
keywords: {', '.join(b['keywords'])}
constraints:
  max_len: {b['constraints']['max_len']}
  allowed_tlds: {', '.join(b['constraints']['allowed_tlds'])}
  forbid_digits: {b['constraints']['forbid_digits']}
  forbid_hyphens: {b['constraints']['forbid_hyphens']}
  ascii_only: {b['constraints']['ascii_only']}"""

        # Format chosen/rejected as JSON single-item suggestions to keep sequences short
        def to_json(c):
            return json.dumps({
                "query_id": b["brief_id"],
                "suggestions": [{
                    "domain": c["domain"],
                    "rationale": c["rationale"],
                    "scores": c["scores"],
                    "safety": c["safety"]
                }]
            }, ensure_ascii=False)

        rows.append({"prompt": prompt, "chosen": to_json(w), "rejected": to_json(l)})
    return rows

dpo_data = ds.Dataset.from_list(dpo_rows(pairs))
dpo_data = dpo_data.train_test_split(test_size=0.1, seed=7)
dpo_data


In [ ]:
from trl import DPOTrainer, DPOConfig

# Load the baseline SFT adapter as the starting point (policy init)
policy = AutoModelForCausalLM.from_pretrained(BASE_MODEL, quantization_config=bnb_cfg, device_map="auto", torch_dtype=torch.bfloat16)
policy = get_peft_model(policy, peft_cfg)  # same config as baseline
policy.load_adapter(OUTPUT_DIR)            # load LoRA weights from baseline

dpo_args = DPOConfig(
    output_dir="/content/drive/MyDrive/domain_suggest/checkpoints/dpo_v1",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=5e-6,
    num_train_epochs=1,
    beta=0.1,  # DPO temperature
    logging_steps=20,
    save_strategy="epoch",
    eval_strategy="no",
    bf16=True,
    report_to="none"
)

dpo_trainer = DPOTrainer(
    model=policy,
    args=dpo_args,
    beta=dpo_args.beta,
    train_dataset=dpo_data["train"],
    eval_dataset=None,
    tokenizer=tok,
    max_length=1792,
    max_target_length=384,
    max_prompt_length=1024
)
dpo_trainer.train()
dpo_trainer.save_model("/content/drive/MyDrive/domain_suggest/checkpoints/dpo_v1")
tok.save_pretrained("/content/drive/MyDrive/domain_suggest/checkpoints/dpo_v1")


In [ ]:
import json, time, os
def record_version(path, tag, parent, notes, metrics):
    info = {
        "tag": tag,
        "timestamp": time.time(),
        "parent": parent,
        "notes": notes,
        "metrics": metrics
    }
    with open(os.path.join(path, "model_version.json"), "w") as f:
        json.dump(info, f, indent=2)

# Example:
record_version("/content/drive/MyDrive/domain_suggest/checkpoints/baseline_qlora", "v0.1-baseline-qlora", None, "Initial SFT QLoRA", {"val_loss": trainer.state.log_history[-1].get("eval_loss", None)})


In [ ]:
from huggingface_hub import HfApi, create_repo, upload_folder

org_or_user = "your-hf-username"
repo_name = "domain-suggester-qwen25-3b"
repo_id = f"{org_or_user}/{repo_name}"

# Create once:
# create_repo(repo_id, private=True)

# Upload each run:
api = HfApi()
api.upload_folder(
    folder_path="/content/drive/MyDrive/domain_suggest/checkpoints/baseline_qlora",
    repo_id=repo_id,
    path_in_repo="v0.1-baseline-qlora",
)
# Add a tag by creating a release or using "refs/tags/<tag>" via git if you clone the repo.


In [ ]:
# LLM-as-a-Judge Evaluation Framework (Colab-ready files)
# - Creates a reusable module with rubric, rule checks, and multiple judge backends:
#   * RuleBasedJudge (works offline; useful as a fallback & sanity check)
#   * OpenAIJudge / AnthropicJudge stubs (for API-based LLM evaluation)
#   * LocalHFJudge (for open-source models if you want to run locally with Transformers)
# - Includes a runner script that evaluates a dataset (from Step 1) and writes:
#   * evaluation_report.jsonl (per-suggestion breakdown)
#   * evaluation_summary.md (aggregated metrics)
#
# In this environment we cannot call external APIs or download models,
# so the demo below will run the RuleBasedJudge on your dataset.
#
# You can download and use these files directly in Colab.

import os, json, math, statistics, re, uuid, random, textwrap
from datetime import datetime
import pandas as pd

BASE_DIR = "/mnt/data/llm_judge_framework"
os.makedirs(BASE_DIR, exist_ok=True)

# ------------------------------
# 1) llm_judge.py (framework)
# ------------------------------
llm_judge_py = r'''
"""
llm_judge.py — LLM-as-a-Judge framework for domain name suggestions

Backends:
- RuleBasedJudge: offline heuristic scorer (baseline + sanity check)
- OpenAIJudge: calls OpenAI Chat Completions API (requires OPENAI_API_KEY)
- AnthropicJudge: calls Anthropic Messages API (requires ANTHROPIC_API_KEY)
- LocalHFJudge: runs an open-source judge with transformers (optional)

Outputs:
- Per-suggestion JSON records with rubric scores and a weighted final_score.
- Aggregated summary metrics helper.

NOTE: Availability/trademark checks are out of scope for this judge.
"""

from __future__ import annotations
import os, json, re, math, random, statistics, time
from dataclasses import dataclass
from typing import List, Dict, Any, Optional, Tuple

# --------------------
# Rubric & weighting
# --------------------

RUBRIC_WEIGHTS = {
    # Gating: if "safety" or "spec_compliance" fails (0), final_score = 0
    "brandability": 0.20,
    "relevance": 0.20,         # keyword/industry fit
    "pronounceability": 0.15,
    "memorability": 0.10,
    "novelty": 0.10,
    "clarity": 0.10,           # avoids ambiguity/confusables
    "tld_fit": 0.05,
    "structure_quality": 0.10  # JSON formatting & rationale helpfulness
}

@dataclass
class Brief:
    brief_id: str
    title: str
    language: str
    tone: str
    keywords: List[str]
    constraints: Dict[str, Any]
    notes: str = ""

@dataclass
class Candidate:
    candidate_id: str
    brief_id: str
    domain: str
    rationale: str
    passes_constraints: bool
    safety: Dict[str, Any]

# --------------------
# Helper: spec checks
# --------------------

VOWELS = set("aeiou")
AMBIGUOUS = set(list("l1I0O5S"))

def spec_checks(brief: Brief, domain: str) -> Tuple[bool, List[str]]:
    reasons = []
    name, *rest = domain.split(".")
    max_len = brief.constraints.get("max_len", 12)
    allowed = brief.constraints.get("allowed_tlds", [])
    forbid_digits = brief.constraints.get("forbid_digits", True)
    forbid_hyphens = brief.constraints.get("forbid_hyphens", True)
    ascii_only = brief.constraints.get("ascii_only", True)

    if len(name) > max_len:
        reasons.append("length_exceeded")
    if forbid_digits and any(ch.isdigit() for ch in name):
        reasons.append("digits_forbidden")
    if forbid_hyphens and "-" in name:
        reasons.append("hyphen_forbidden")
    if ascii_only and not name.isascii():
        reasons.append("non_ascii")
    if allowed:
        tld = "." + domain.split(".")[-1]
        if tld not in allowed:
            reasons.append("tld_not_allowed")
    ok = len(reasons) == 0
    return ok, reasons

# --------------------
# Heuristic proxies
# --------------------

def pronounceability_score(name: str) -> float:
    # Simple heuristic: penalize long consonant runs; reward vowel presence
    if not name: return 0.0
    runs = re.findall(r"[^aeiou]+", name)
    max_run = max((len(r) for r in runs), default=0)
    vowel_ratio = sum(1 for c in name if c in VOWELS) / max(1, len(name))
    score = 0.6 * (1 - min(max_run/5, 1)) + 0.4 * min(vowel_ratio/0.4, 1)
    return max(0.0, min(1.0, score))

def memorability_score(name: str) -> float:
    # Shorter, with some repetition but not too much
    if not name: return 0.0
    length = len(name)
    unique_ratio = len(set(name))/length
    repeat_penalty = 1 - abs(unique_ratio - 0.7)  # prefer ~0.7 unique ratio
    base = max(0.0, 1 - (length - 6)/10)  # 6..16
    score = 0.6*base + 0.4*repeat_penalty
    return max(0.0, min(1.0, score))

def clarity_score(name: str) -> float:
    # Penalize ambiguous chars (l/1/I, 0/O, 5/S)
    amb_count = sum(1 for c in name if c in AMBIGUOUS)
    score = max(0.0, 1 - amb_count / max(4, len(name)/2))
    return max(0.0, min(1.0, score))

def novelty_score(name: str, keywords: List[str]) -> float:
    # Prefer candidates that are not raw keywords; mild penalty for substring overlap
    n = name.lower()
    overlaps = sum(1 for k in keywords if k.lower() in n)
    return max(0.0, min(1.0, 1 - overlaps*0.3))

def relevance_score(name: str, keywords: List[str], rationale: str) -> float:
    # Proxy: keyword substrings in rationale OR name boosts score
    text = (name + " " + rationale).lower()
    hits = sum(1 for k in keywords if k.lower() in text)
    return max(0.0, min(1.0, hits / max(1, len(keywords))))

def brandability_score(name: str) -> float:
    # Blend of pronounceability + memorability + absence of digits/hyphen
    base = 0.5*pronounceability_score(name) + 0.5*memorability_score(name)
    if any(ch.isdigit() for ch in name) or "-" in name:
        base *= 0.7
    return max(0.0, min(1.0, base))

def tld_fit_score(domain: str, allowed_tlds: List[str]) -> float:
    if not allowed_tlds: return 1.0
    tld = "." + domain.split(".")[-1]
    return 1.0 if tld in allowed_tlds else 0.0

def structure_quality_score(rationale: str) -> float:
    # Very rough: presence of short justification, no obvious profanity (not exhaustive)
    txt = rationale.strip().lower()
    if not txt: return 0.2
    return max(0.2, min(1.0, 0.5 + min(len(txt), 200)/400))

# --------------------
# Judges
# --------------------

class BaseJudge:
    def score_candidate(self, brief: Brief, cand: Candidate) -> Dict[str, Any]:
        raise NotImplementedError

    @staticmethod
    def aggregate_scores(scores: Dict[str, float], gates: Dict[str, bool]) -> float:
        if not gates.get("safety_ok", True): return 0.0
        if not gates.get("spec_ok", True): return 0.0
        total = 0.0
        for k, w in RUBRIC_WEIGHTS.items():
            total += w * max(0.0, min(1.0, scores.get(k, 0.0)))
        return round(total, 4)

class RuleBasedJudge(BaseJudge):
    def score_candidate(self, brief: Brief, cand: Candidate) -> Dict[str, Any]:
        name = cand.domain.split(".")[0].lower()
        spec_ok, spec_reasons = spec_checks(brief, cand.domain)
        safety_ok = not cand.safety.get("flagged", False)

        scores = {
            "brandability": brandability_score(name),
            "relevance": relevance_score(name, brief.keywords, cand.rationale),
            "pronounceability": pronounceability_score(name),
            "memorability": memorability_score(name),
            "novelty": novelty_score(name, brief.keywords),
            "clarity": clarity_score(name),
            "tld_fit": tld_fit_score(cand.domain, brief.constraints.get("allowed_tlds", [])),
            "structure_quality": structure_quality_score(cand.rationale),
        }
        gates = {"safety_ok": safety_ok, "spec_ok": spec_ok}
        final = self.aggregate_scores(scores, gates)
        return {
            "scores": scores,
            "gates": gates,
            "final_score": final,
            "spec_reasons": spec_reasons,
            "safety_reasons": cand.safety.get("reasons", []),
            "judge_backend": "rule_based"
        }

# --- Placeholders for real LLM judges ---

class OpenAIJudge(BaseJudge):
    """
    Usage:
        judge = OpenAIJudge(model="gpt-4o-mini", api_key=os.environ["OPENAI_API_KEY"])
        result = judge.score_candidate(brief, cand)
    """
    def __init__(self, model: str, api_key: Optional[str] = None):
        self.model = model
        self.api_key = api_key or os.getenv("OPENAI_API_KEY")

    def _build_prompt(self, brief: Brief, cand: Candidate) -> list:
        system = (
            "You are an expert brand evaluator. Score a proposed domain name using the rubric."
            " Refuse to endorse unsafe/inappropriate content. Do NOT reveal chain-of-thought."
            " Return strict JSON with numeric scores 0..1 and one-sentence reasons."
        )
        user = f"""
[BRIEF]
title: {brief.title}
language: {brief.language}
tone: {brief.tone}
keywords: {", ".join(brief.keywords)}
constraints:
  max_len: {brief.constraints.get("max_len")}
  allowed_tlds: {", ".join(brief.constraints.get("allowed_tlds", []))}
  forbid_digits: {brief.constraints.get("forbid_digits")}
  forbid_hyphens: {brief.constraints.get("forbid_hyphens")}
  ascii_only: {brief.constraints.get("ascii_only")}

[CANDIDATE]
domain: {cand.domain}
rationale: {cand.rationale}

[OUTPUT_JSON_SCHEMA]
{{
  "gates": {{"safety_ok": true, "spec_ok": true}},
  "scores": {{
    "brandability": 0.0, "relevance": 0.0, "pronounceability": 0.0, "memorability": 0.0,
    "novelty": 0.0, "clarity": 0.0, "tld_fit": 0.0, "structure_quality": 0.0
  }},
  "reasons": {{
    "brandability": "", "relevance": "", "pronounceability": "", "memorability": "",
    "novelty": "", "clarity": "", "tld_fit": "", "structure_quality": "", "safety": "", "spec": ""
  }}
}}
Only output JSON, nothing else.
"""
        return [
            {"role": "system", "content": system},
            {"role": "user", "content": user.strip()}
        ]

    def score_candidate(self, brief: Brief, cand: Candidate) -> Dict[str, Any]:
        import requests, json as _json
        url = "https://api.openai.com/v1/chat/completions"
        headers = {"Authorization": f"Bearer {self.api_key}", "Content-Type": "application/json"}
        payload = {
            "model": self.model,
            "messages": self._build_prompt(brief, cand),
            "temperature": 0.2
        }
        resp = requests.post(url, headers=headers, json=payload, timeout=60)
        resp.raise_for_status()
        content = resp.json()["choices"][0]["message"]["content"]
        try:
            out = _json.loads(content)
        except Exception:
            out = {"gates": {"safety_ok": True, "spec_ok": True}, "scores": {}, "reasons": {"parse_error": content}}
        # Aggregate
        final = BaseJudge.aggregate_scores(out.get("scores", {}), out.get("gates", {}))
        out["final_score"] = final
        out["judge_backend"] = "openai"
        return out

class AnthropicJudge(BaseJudge):
    """
    Usage:
        judge = AnthropicJudge(model="claude-3-5-sonnet", api_key=os.environ["ANTHROPIC_API_KEY"])
    """
    def __init__(self, model: str, api_key: Optional[str] = None):
        self.model = model
        self.api_key = api_key or os.getenv("ANTHROPIC_API_KEY")

    def score_candidate(self, brief: Brief, cand: Candidate) -> Dict[str, Any]:
        import requests, json as _json
        url = "https://api.anthropic.com/v1/messages"
        headers = {
            "x-api-key": self.api_key,
            "anthropic-version": "2023-06-01",
            "content-type": "application/json"
        }
        system = "You are an expert brand evaluator... (same policy as OpenAIJudge)."
        user = "Same formatted prompt as in OpenAIJudge._build_prompt"
        payload = {
            "model": self.model,
            "system": system,
            "messages": [{"role":"user","content": user}],
            "max_tokens": 400,
            "temperature": 0.2
        }
        resp = requests.post(url, headers=headers, json=payload, timeout=60)
        resp.raise_for_status()
        text = resp.json()["content"][0]["text"]
        try:
            out = _json.loads(text)
        except Exception:
            out = {"gates": {"safety_ok": True, "spec_ok": True}, "scores": {}, "reasons": {"parse_error": text}}
        final = BaseJudge.aggregate_scores(out.get("scores", {}), out.get("gates", {}))
        out["final_score"] = final
        out["judge_backend"] = "anthropic"
        return out

class LocalHFJudge(BaseJudge):
    """
    Use a local open-source model as judge (e.g., Qwen2.5-3B-Instruct).
    Requires transformers + bitsandbytes; not suitable for CPU-only.
    """
    def __init__(self, model_id: str = "Qwen/Qwen2.5-3B-Instruct"):
        from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
        import torch
        bnb_cfg = BitsAndBytesConfig(
            load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
        self.tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
        self.model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_cfg, device_map="auto")
        self.model.eval()

    def _prompt(self, brief: Brief, cand: Candidate) -> str:
        system = "You are an expert brand evaluator. Return only JSON with scores 0..1 and one-sentence reasons."
        user = f"""
[BRIEF]
title: {brief.title}
language: {brief.language}
tone: {brief.tone}
keywords: {", ".join(brief.keywords)}
constraints: {brief.constraints}

[CANDIDATE]
domain: {cand.domain}
rationale: {cand.rationale}
"""
        return f"<|im_start|>system\n{system}\n<|im_end|>\n<|im_start|>user\n{user}\n<|im_end|>\n<|im_start|>assistant\n"

    def score_candidate(self, brief: Brief, cand: Candidate) -> Dict[str, Any]:
        import torch, json as _json
        text = self._prompt(brief, cand)
        inputs = self.tok(text, return_tensors="pt").to(self.model.device)
        with torch.no_grad():
            out_ids = self.model.generate(**inputs, max_new_tokens=320, do_sample=False, temperature=0.0)
        out = self.tok.decode(out_ids[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
        try:
            data = _json.loads(out)
        except Exception:
            data = {"gates": {"safety_ok": True, "spec_ok": True}, "scores": {}, "reasons": {"parse_error": out}}
        final = BaseJudge.aggregate_scores(data.get("scores", {}), data.get("gates", {}))
        data["final_score"] = final
        data["judge_backend"] = "local_hf"
        return data

# --------------------
# Evaluation driver
# --------------------

def aggregate_report(rows: List[Dict[str, Any]]) -> Dict[str, Any]:
    finals = [r["final_score"] for r in rows]
    by_gate_fail = {
        "safety_fail": sum(1 for r in rows if not r["gates"]["safety_ok"]),
        "spec_fail": sum(1 for r in rows if not r["gates"]["spec_ok"]),
    }
    summary = {
        "count": len(rows),
        "final_score_mean": round(float(statistics.mean(finals)), 4) if finals else 0.0,
        "final_score_median": round(float(statistics.median(finals)), 4) if finals else 0.0,
        "final_score_p90": round(float(sorted(finals)[int(0.9*(len(finals)-1))]), 4) if finals else 0.0,
        **by_gate_fail
    }
    return summary

def make_brief(obj: Dict[str, Any]) -> Brief:
    return Brief(
        brief_id=obj["brief_id"],
        title=obj["title"],
        language=obj.get("language","en"),
        tone=obj.get("tone",""),
        keywords=obj.get("keywords",[]),
        constraints=obj.get("constraints",{}),
        notes=obj.get("notes","")
    )

def make_candidate(obj: Dict[str, Any]) -> Candidate:
    return Candidate(
        candidate_id=obj["candidate_id"],
        brief_id=obj["brief_id"],
        domain=obj["domain"],
        rationale=obj.get("rationale",""),
        passes_constraints=obj.get("passes_constraints", True),
        safety=obj.get("safety", {"flagged": False, "reasons": []})
    )
'''
with open(os.path.join(BASE_DIR, "llm_judge.py"), "w", encoding="utf-8") as f:
    f.write(llm_judge_py)

# ------------------------------
# 2) run_judge_demo.py (uses RuleBasedJudge now)
# ------------------------------
run_demo_py = r'''
"""
run_judge_demo.py — Demo runner for the LLM-as-a-Judge framework.

Usage (Colab):
    !python run_judge_demo.py --data_dir /content/data --out_dir /content/eval --backend rule
Backends:
    rule  -> RuleBasedJudge (offline)
    openai-> OpenAIJudge (requires OPENAI_API_KEY)
    anthropic -> AnthropicJudge (requires ANTHROPIC_API_KEY)
    local -> LocalHFJudge (loads open-source model; needs GPU & internet to download)
"""

import os, json, argparse, uuid
from datetime import datetime
from collections import defaultdict

from llm_judge import (
    RuleBasedJudge, OpenAIJudge, AnthropicJudge, LocalHFJudge,
    make_brief, make_candidate, aggregate_report, RUBRIC_WEIGHTS
)

def load_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                yield json.loads(line)

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--data_dir", type=str, required=True, help="Folder containing domain_briefs.jsonl and domain_candidates.jsonl")
    ap.add_argument("--out_dir", type=str, required=True)
    ap.add_argument("--backend", type=str, default="rule", choices=["rule","openai","anthropic","local"])
    ap.add_argument("--max_per_brief", type=int, default=10)
    args = ap.parse_args()

    briefs = list(load_jsonl(os.path.join(args.data_dir, "domain_briefs.jsonl")))
    cands  = list(load_jsonl(os.path.join(args.data_dir, "domain_candidates.jsonl")))

    brief_map = {b["brief_id"]: make_brief(b) for b in briefs}
    by_brief = defaultdict(list)
    for c in cands:
        by_brief[c["brief_id"]].append(make_candidate(c))

    # Select judge
    if args.backend == "rule":
        judge = RuleBasedJudge()
    elif args.backend == "openai":
        judge = OpenAIJudge(model=os.getenv("OPENAI_MODEL","gpt-4o-mini"))
    elif args.backend == "anthropic":
        judge = AnthropicJudge(model=os.getenv("ANTHROPIC_MODEL","claude-3-5-sonnet"))
    else:
        judge = LocalHFJudge(model_id=os.getenv("LOCAL_JUDGE_MODEL","Qwen/Qwen2.5-3B-Instruct"))

    os.makedirs(args.out_dir, exist_ok=True)
    report_path = os.path.join(args.out_dir, "evaluation_report.jsonl")
    summary_path = os.path.join(args.out_dir, "evaluation_summary.md")

    rows = []
    for b in briefs:
        brief = brief_map[b["brief_id"]]
        # take up to N candidates per brief (prioritize ones that passed constraints in dataset)
        cand_list = sorted(by_brief[brief.brief_id], key=lambda x: (x.passes_constraints, not x.safety.get("flagged", False)), reverse=True)[:args.max_per_brief]
        for cand in cand_list:
            scored = judge.score_candidate(brief, cand)
            rows.append({
                "eval_id": str(uuid.uuid4()),
                "timestamp": datetime.utcnow().isoformat()+"Z",
                "brief_id": brief.brief_id,
                "brief_title": brief.title,
                "domain": cand.domain,
                "scores": scored["scores"],
                "gates": scored["gates"],
                "final_score": scored["final_score"],
                "spec_reasons": scored.get("spec_reasons", []),
                "safety_reasons": scored.get("safety_reasons", []),
                "backend": scored.get("judge_backend", "unknown")
            })

    with open(report_path, "w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r)+"\n")

    # Aggregate
    overall = aggregate_report(rows)

    # Per-brief top picks
    top_by_brief = defaultdict(list)
    for r in rows:
        top_by_brief[r["brief_id"]].append(r)
    for k in top_by_brief:
        top_by_brief[k].sort(key=lambda x: x["final_score"], reverse=True)

    # Write human-readable summary
    with open(summary_path, "w", encoding="utf-8") as f:
        f.write(f"# Evaluation Summary\n\n")
        f.write(f"Generated: {datetime.utcnow().isoformat()}Z\n\n")
        f.write(f"**Backend:** `{args.backend}`  \n")
        f.write(f"**Rubric weights:** `{RUBRIC_WEIGHTS}`\n\n")
        f.write(f"## Overall Metrics\n")
        for k, v in overall.items():
            f.write(f"- **{k}**: {v}\n")
        f.write("\n## Top picks per brief\n")
        for b in briefs:
            arr = top_by_brief[b['brief_id']][:3]
            f.write(f"\n### {b['title']}\n")
            for r in arr:
                f.write(f"- `{r['domain']}` — score {r['final_score']:.3f} (gates: {r['gates']})\n")

    print(f"Wrote:\n- {report_path}\n- {summary_path}")
    return 0

if __name__ == "__main__":
    raise SystemExit(main())
'''
with open(os.path.join(BASE_DIR, "run_judge_demo.py"), "w", encoding="utf-8") as f:
    f.write(run_demo_py)

# ------------------------------
# 3) Create a minimal demo run using the dataset from earlier step (if present)
# ------------------------------
DATA_DIR = "/mnt/data/domain_dataset_v1"
OUT_DIR = os.path.join(BASE_DIR, "demo_eval")
os.makedirs(OUT_DIR, exist_ok=True)

# Check dataset presence
has_dataset = all(os.path.exists(os.path.join(DATA_DIR, fn)) for fn in ["domain_briefs.jsonl","domain_candidates.jsonl"])

demo_note = {}
if has_dataset:
    # Run the rule-based judge to produce a report and summary
    import subprocess, sys, textwrap
    cmd = ["python", os.path.join(BASE_DIR, "run_judge_demo.py"),
           "--data_dir", DATA_DIR, "--out_dir", OUT_DIR, "--backend", "rule", "--max_per_brief", "8"]
    try:
        subprocess.check_call(cmd)
        demo_note["ran_demo"] = True
    except Exception as e:
        demo_note["ran_demo"] = False
        demo_note["error"] = str(e)
else:
    demo_note["ran_demo"] = False
    demo_note["error"] = "Dataset from Step 1 not found at /mnt/data/domain_dataset_v1"

# Show what we created
files = {
    "module": os.path.join(BASE_DIR, "llm_judge.py"),
    "runner": os.path.join(BASE_DIR, "run_judge_demo.py"),
    "report": os.path.join(OUT_DIR, "evaluation_report.jsonl"),
    "summary": os.path.join(OUT_DIR, "evaluation_summary.md")
}

files, demo_note
